# Library

In [ ]:
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras. applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix ,accuracy_score,cohen_kappa_score,roc_curve,auc,recall_score,precision_score,f1_score
from keras import optimizers

# Dataset configuration

In [ ]:
train_path = '../Documents/Bases/Melanoma/train'
test_path = '../Documents/Bases/Melanoma/test'
num_classes = 8

# Loading training data

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(224,224),color_mode='rgb',batch_size=64,shuffle=True)

# Creating the model

In [ ]:
model = VGG16(weights='imagenet', include_top=True)
model.layers.pop()
new_layer = Dense(num_classes, activation='softmax', name='predictions')
model = Model(model.input, new_layer(model.layers[-1].output))

In [ ]:
sgd = optimizers.SGD(lr=0.0001,decay=0.0001)
model.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [ ]:
history = model.fit_generator(train_batches,epochs=2,verbose=1)

# Prediction

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_path,batch_size=64,shuffle=False,target_size=(224,224))
predicts_vectors = model.predict_generator(test_generator, verbose=1)

# Validation

In [ ]:
pred = np.argmax(predicts_vectors,axis=1)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer () 

lb.fit(test_generator.labels)

y_test = lb.transform (test_generator.labels)
y_pred = lb.transform (pred)

auc_score = roc_auc_score(y_test, y_pred,average="macro")

In [ ]:
y_true = test_generator.labels
y_pred = pred

recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
kappa = cohen_kappa_score(test_generator.labels, pred)
acc = accuracy_score(test_generator.labels, pred)

In [ ]:
print('>> Resultados <<')
print('Acc: {}'.format(acc*100))
print('Recall: {}'.format(recall*100))
print('Precision: {}'.format(precision*100))
print('F1-Acore: {}'.format(f1))
print('AUC: {}'.format(auc_score))
print('Kappa: {}'.format(kappa))